# Communication using Twilio and email

- Some of the concepts here can be found in more depth on
    - https://www.amazon.com/Data-Wrangling-Python-Tools-Easier/dp/1491948817/
    - Free on the UCSF network: http://proquest.safaribooksonline.com/book/databases/9781491948804

### Libaries we need

In [ ]:
import os
import pandas as pd
import time
import pickle
from datetime import datetime

# Twilio library
from twilio.rest import Client

# email libraries
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication

# Custom libraries
import common.phone_utils as pu
from redcap.redcapy import Redcapy
from common.sms_admin import SMSAdmin
from common.logs import Logs
from common.twilio_helper import TwilioHelper

## Twilio
- API Documentation: https://www.twilio.com/docs/sms/api
- To fully explore the capabilities of Twilio, please refer to documentation and/or call Twilio sales
- Please refer to presentation slides for more details


### Documenting Production Code Execution

For running production code unattended, it is important to document key aspects of your code execution.  
- Useful for auditing purposes
- Useful for troubleshooting

- If executing a notebook, you can save a copy of the notebook.

- If executing a script, you can:
 - Print to standard output, and results will be saved in mail (not ideal, but better than nothing)
 - Save select results to a log
 
Even for notebook execution, a log is helpful for searching in one place through the complete history of that code's execution


### Setup a logging facility
- This example uses a custom library (imported above) than handles some basics, so the use of a log is relatively straightforward for any program
- For a complete understanding of how to use logging, please review external documentation.  Recommendations:
 - https://fangpenlin.com/posts/2012/08/26/good-logging-practice-in-python/
 - https://docs.python.org/3.7/library/logging.html
 
- As we execute code in this notebook, we can check the log in a terminal window
 - A useful way to monitor changes to any file, such as a log, can be done with the linux command tail -f
 - We will return to this log as we execute code later

In [ ]:
log_dir = 'presentation_logs'
log_instance = Logs(log_dir=log_dir, log_filename='caps_{}.log'.format('copy'), level='DEBUG')
logger = log_instance.logger

### Set Pandas options

In [ ]:
pd.options.display.max_columns = 200
pd.options.display.max_rows = 200

### Initialize SMTP to Send Email
- UCSF Reference: https://it.ucsf.edu/services/site-email/tutorial/ucsf-email-pop-and-imap-settings?page=show
- References
    - https://stackoverflow.com/questions/3362600/how-to-send-email-attachments
    - https://stackoverflow.com/questions/31433633/reply-to-email-using-python-3-4
    - https://stackoverflow.com/questions/24672079/send-email-using-smtp-ssl-port-465
- https://stackoverflow.com/questions/16968758/sending-email-to-a-microsoft-exchange-group-using-python

In [ ]:
# Common Elements
email_server_name = 'smtp.office365.com'
email_server_port = '587'
use_ssl = True
address_book = [os.environ['BEECON3_EMAIL_RECIPIENTS_TEST']]  # list of addresses
sender = os.environ['SERVER_EMAIL_ID']
sender_pw = os.environ['SERVER_EMAIL_PW']

def send_email(subject, body):
    msg = MIMEMultipart()    

    # email_server = email_server_name if not email_server_port else email_server_name + email_server_port
    email_server = ':'.join([email_server_name, email_server_port])

    msg['From'] = sender
    msg['Reply-To'] = sender
    msg['To'] = ','.join(address_book)
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'plain'))
    text = msg.as_string()

    # print(text)
    try:
        s = smtplib.SMTP(email_server)
        s.starttls() if use_ssl else None
        s.login(sender, sender_pw)
        s.sendmail(sender, address_book, msg.as_string())
        print('Email sent to {}'.format(', '.join(address_book)))
    except Exception as e:
        smtp_msg = 'Unable to Send email.  Error: {}'.format(e)
        print(smtp_msg)
    finally:
        s.quit() if 's' in locals() else None    

### Initialize Twilio
- Download the helper library from https://www.twilio.com/docs/python/install

In [ ]:
# Your Account Sid and Auth Token from twilio.com/console
account_sid = os.environ['TWILIO_SID']
auth_token = os.environ['TWILIO_TOKEN']
client = Client(account_sid, auth_token)

## Objective: Send Follow Up Message to Participants Who Completed 6 Month Exam 

To do this, we need to filter out those who have completed the 12 Month Exam already.

Normally, a task such as this would occur within a defined window after the visit occurred

### First, restore our DataFrame from the Redcapy notebook demo

In [ ]:
%store -r caps_combined_df
combined_df = caps_combined_df
combined_df

### Create a list of those who completed the 12 month visit

In [ ]:
id_to_m12_complete = list(combined_df[(combined_df.exam_complete == '2') & 
                    ((combined_df.redcap_event_name == '12_month_arm_2') | 
                     (combined_df.redcap_event_name == '12_month_arm_3'))]['record_id'].values)
id_to_m12_complete

### Filter the DataFrame to those who completed the 6M exam but not the 12M exam
- Note that multiple conditions in a DataFrame filter require parentheses and & | operators.
- Negation is done with a ~ operator
- When checking if elements of a series are contained in a list, use isin()

In [ ]:
m6_complete_df = combined_df[(combined_df.exam_complete == '2') & 
                             ((combined_df.redcap_event_name == '6_month_arm_2') | 
                              (combined_df.redcap_event_name == '6_month_arm_3')) &
                             ~combined_df.record_id.isin(id_to_m12_complete)
                            ]
m6_complete_df

### Create a dict of IDs to phone numbers
- Recall phone numbers are recorded at the baseline event.  Hence, they are not contained in the 6M DataFrame
- Here, we use a combination of dict and zip to pair corresponding ids and numbers from the DF to a dict
- zip ref: https://stackoverflow.com/questions/13704860/zip-lists-in-python

In [ ]:
baseline_df = combined_df[combined_df.redcap_event_name == 'baseline_arm_1']
id_to_mobile = dict(zip(baseline_df.record_id, baseline_df.part_mobile_number))
id_to_mobile

### Similarly with names

In [ ]:
id_to_first_name = dict(zip(baseline_df.record_id, baseline_df.part_first_name))
id_to_first_name

### Set up a directory for pickling the status DataFrame

In [ ]:
pickle_dir = os.getcwd()
pickle_file = 'sms_df.pickle'

try:
    os.makedirs(pickle_dir) if not os.path.exists(pickle_dir) else None
    pickle_file_full_path = os.path.join(pickle_dir, pickle_file)

except Exception as e:
    msg = 'Unable to create or access pickle directory: {}.  Error: {}'.format(pickle_dir, e)
    logger.error(msg)

### Function to convert Twilio datetime objects to local timezone
- We can use an environment variable to set the datetime
- https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior
- http://strftime.org/

In [ ]:
timezone = os.environ['TIMEZONE_SERVER']

def localize_twilio_dt(dt, tz=timezone):
    if isinstance(dt, datetime):
        return pd.Timestamp(dt).tz_convert(tz)

    else:
        # Add type checking as needed
        print('Failed to convert datetime timezone to {}'.format(tz))
        return dt
        # raise ValueError('Only a datetime objecdt can be localized with this function')
        
timezone

### Send SMS
- This is adapted from the Twilio sample code
    - https://www.twilio.com/docs/sms/send-messages
- Initialize a DataFrame to capture delivery status
- Append to the DataFrame one row at a time using .loc
- Pickle the DataFrame one row at a time if performance is not an issue
    - This technique will allow you to recover the state of messages sent, with SIDs, in the event of code failure and status needs to be rechecked on a subsequent code session
    - By checking delivery, you can avoid resending texts to those who have already received the SMS, while focusing on sending to those who did not receive it on the prior attempt

In [ ]:
message_dict = {} 
status_keys = ['txt_mobile_number',
               'txt_twilio_sid',
               'txt_twilio_error_code',
               'txt_twilio_error_msg',
               'txt_twilio_status',
               'txt_date_created',
               'txt_date_updated',
               'txt_message',               
               'record_id',
               'redcap_event_name',
               'redcap_repeat_instrument',
               'redcap_repeat_instance',
               'sms_survey_log_complete',
              ]

sms_df = pd.DataFrame(columns=status_keys)

for i, row in m6_complete_df.iterrows():
    message = ''
    phone = '+1' + id_to_mobile[row.record_id]
    phone = '+15129202947'  #TODO
    
    name = id_to_first_name[row.record_id]
    
    sms_body = 'Thanks {} for completing the 6 month exam. '.format(name)
    sms_body += 'See you soon at your final visit'    
    
    # Initialize dict to capture status details
    message_dict = {}
    message_dict.update({
        'record_id': row.record_id,
        'redcap_event_name': row.redcap_event_name,
        'redcap_repeat_instrument': row.redcap_repeat_instrument,
        'redcap_repeat_instance': row.redcap_repeat_instance,
        'txt_mobile_number': phone,
        'txt_twilio_sid': '',
        'txt_twilio_error_code': '',
        'txt_twilio_error_msg': '',
        'txt_twilio_status': 'attempting',
        'txt_date_created': pd.Timestamp(datetime.utcnow()).tz_localize('UTC'),
        'txt_date_updated': pd.Timestamp(datetime.utcnow()).tz_localize('UTC'),
        'txt_message': sms_body,
    })    

    time.sleep(2)
    connected = False
    
    try:
        # API Call to TWilio to send SMS
        message = client.messages.create(from_=os.environ['BEECON3_FROM_ADMIN_PHONE'],
                                         body=sms_body,
                                         to=phone,
                                        )
        message_dict.update({
            'txt_twilio_sid': message.sid,
            'txt_twilio_error_code': '' if not message.error_code else message.error_code,
            'txt_twilio_error_msg': '' if not message.error_message else message.error_message,
            'txt_twilio_status': message.status,
            'txt_date_created': localize_twilio_dt(message.date_created),
            'txt_date_updated': localize_twilio_dt(message.date_updated),
        })    
        
        connected = True
    except Exception as e:
        msg = 'ID: {}, Failed to send SMS. Error {}'.format(row.record_id, e)
        logger.error(msg)
        print(msg)
        
    # Note, appending a row to a DataFrame efficiently is tricky
    row_df = pd.DataFrame([message_dict])
    sms_df.loc[i, row_df.columns.tolist()] = row_df.values
    
    # Note pickle restorations will use the undated pickle, but we are saving a datestamped pickle as a backup
    #  in the event a restore is needed from an older overwritten pickle
    try:
        print('Storing sms_df pickle')
        if sms_df.shape[0] > 0:
            pickle.dump(sms_df, open(pickle_file_full_path, 'wb'))
            pickle.dump(sms_df,
                        open(os.path.join(pickle_dir, datetime.now().date().strftime('%Y%m%d') + '_'
                                          + pickle_file), 'wb'))
            
            msg = 'Stored sms_df to pickle file {}'.format(pickle_file_full_path)
            logger.info(msg)
            print(msg)
        else:
            msg = 'sms_df not stored to pickle file due to 0 length'
            logger.info(msg)
            print(msg)
    except Exception as e:
        msg = 'Unable to store sms_df to simple or date-stamped pickle files {}. '.format(
            pickle_file_full_path) 
        msg += 'Continuing code execution. Error returned: {}'.format(e)
        
        logger.error(msg)
    
    msg = 'ID: {}, phone: {}, SID: {}, Attempted 6 Month follow up SMS'.format(row.record_id,
                                                                               row.part_mobile_number,
                                                                               message.sid if message else '',
                                                                              )
    if connected:
        logger.info(msg)
        print(msg)

sms_df

### Check Status of Delivery Attempts
- Normally, you may want to pause before checking status to allow the carrier to update Twilio with delivery status
- You may also want to create a loop to implement several attempts at checking before abandoning
- Note: Twilio timestamps are in UTC time, so you may want to localize the timezone
- Set the form completion status to complete
- Ref: https://www.twilio.com/docs/sms/api/message#fetch-a-message-resource

In [ ]:
time.sleep(1)  # Set to a more reasonable number than 1

for i, row in sms_df.iterrows():
    if row.txt_twilio_sid:
        try:
            tm = client.messages(row.txt_twilio_sid).fetch()  # API Call to Twilio
        except Exception as e:
            msg = 'Unable to connect to Twilio to check delivery status for ID {}'.format(row.record_id)
            logger.error(msg)
            print(msg)
        
        sms_df.loc[i, 'txt_twilio_status'] = tm.status
        sms_df.loc[i, 'txt_date_created'] = localize_twilio_dt(tm.date_created).strftime('%Y-%m-%d %H:%M:%S')        
        sms_df.loc[i, 'txt_date_updated'] = localize_twilio_dt(tm.date_updated).strftime('%Y-%m-%d %H:%M:%S')
        sms_df.loc[i, 'txt_twilio_error_code'] = '' if not tm.error_code else tm.error_code 
        sms_df.loc[i, 'txt_twilio_error_msg'] = '' if not tm.error_message else tm.error_message
        sms_df.loc[i, 'sms_survey_log_complete'] = '2'

sms_df

### DataFrame is now suitable for import into Redcap

In [ ]:
redcap_token = os.environ['REDCAP_API_CAPS_DEMO']
redcap_url = os.environ['REDCAP_URL']

rci = Redcapy(api_token=redcap_token, redcap_url=redcap_url)

### Redcap Import function
- Now includes the current timestamp on import
- Wrapped import in try/except block
- Added logging

In [ ]:
def import_to_redcap(redcap_instance, df_to_upload, overwrite=False):
    import_success_count = 0
    import_attempt_count = 0
    
    overwrite_behavior = 'overwrite' if overwrite else 'normal'
    
    for i, row in df_to_upload.iterrows():
        row['txt_last_timestamp'] = datetime.now().strftime('%Y/%m/%d %H:%M:%S')
        record_to_upload = row.to_json(orient='columns')
        
        try:
            import_attempt_count += 1

            # Below will return {'count': 1} if successful
            import_return = redcap_instance.import_records(data_to_upload=record_to_upload, 
                                                           overwriteBehavior=overwrite_behavior)  
            import_success_count += 1 if 'count' in import_return and import_return['count'] == 1 else 0
            
            msg = 'Imported record id {}, event {} to Redcap'.format(row.record_id,
                                                                     row.redcap_event_name)
            logger.info(msg)
        except Exception as e:
            msg = 'Unable to import record id {}, event {} to Redcap.  Error {}'.format(row.record_id,
                                                                                        row.redcap_event_name,
                                                                                        e
                                                                                       )
            logger.error(msg)
            print(msg)
    
    subject = ('Demo Import Succeeded' if import_attempt_count == import_success_count 
               else 'Error on Demo Import')
    body = 'Imported {} of {} attempts at {}'.format(import_success_count,
                                                     import_attempt_count,
                                                     datetime.now().strftime('%Y/%m/%d %H:%M:%S')
                                                    )
    send_email(subject=subject, body=body)
                    
    return import_success_count, import_attempt_count

In [ ]:
success_count, total_count = import_to_redcap(rci, sms_df)
success_count, total_count, success_count == total_count

# Congratulations!

You are now on the path to:
- Communicate directly with study participants via email or SMS
- Record the status of participant communications in Redcap
- Comminicate reports or status with research staff, investigators, project manager, or yourself
- Save the state of Python objects using a pickle or %store to
    - Recover data from a prior session
    - Share data across notebooks
    - Conserve Redcap server resources
- Log code execution status and errors and review the logs
- Learn more Python, or tackle the challenge of adapting these concepts to R or SAS